### Challenge questions

Easy questions:

 1. How many total pings are in the Ocearch shark data?
 2. How many unique species of sharks are in the data set?
 3. What is the name, weight, and species of the heaviest shark(s)?
 4. When and where was the very first ping?
 5. Excluding results with 0 distance traveled: what's the minimum, average, and maximum travel distances?
 
Intermediate questions:

 1. Which shark had the most pings?
 2. Which shark has been pinging the longest, and how long has that been?
 3. Which shark species has the most individual sharks tagged?
 4. What is the average length and weight of each shark species?
 5. Which shark has the biggest geographic box (largest distance from min lat/lon to max lat/lon, not dist_traveled)?
 
Hard questions:
 1. Use folium to plot the first ping, last ping, and a line connecting each ping for the Tiger shark Emma.  Make the first ping marker a 'play' icon, and last ping icon a 'stop' icon.
 2. Resample Emma data to have a daily lat/lon average, and interpolate missing results.  Plot a marker for each day, and color them blue for hard data, green for interpolated lat/lons
 3. Resample all shark data for daily lat/lon averages, and interpolate missing results
 4. Calculate distance between Emma and other sharks on a daily basis
 5. Identify the shark that has the shortest average distance to Emma per day (minimum 50 days of pings with Emma)
 6. Plot Emma and her closest buddy: interpolated results for each in green, Emma as circle icons and her buddy as square icons

### Load data

#### Query Ocearch API

In [1]:
import requests
url = 'http://www.ocearch.org/tracker/ajax/filter-sharks'

resp = requests.get(url)
resp

<Response [200]>

##### Turn json into dataframe

In [2]:
import pandas as pd
df = pd.DataFrame(resp.json())
columns = ['id', 'name', 'gender', 'species', 'weight', 'length', 'tagDate', 'dist_total']
df[columns].head()

,id,name,gender,species,weight,length,tagDate,dist_total
0,3,Oprah,Female,White Shark (Carcharodon carcharias),686 lb,9 ft 10 in.,7 March 2012,2816.662
1,4,Albertina,Female,White Shark (Carcharodon carcharias),1110 lb,11 ft 6 in.,8 March 2012,1830.593
2,5,Helen,Female,White Shark (Carcharodon carcharias),765 lb,10 ft 2 in.,8 March 2012,4436.661
3,6,Brenda,Female,White Shark (Carcharodon carcharias),1310 lb,12 ft 2 in.,8 March 2012,2966.902
4,7,Madiba,Male,White Shark (Carcharodon carcharias),659 lb,9 ft 8 in.,8 March 2012,3537.423


In [3]:
df.shape

(275, 30)

##### Filter out non-shark data

In [4]:
df.species.value_counts()

Tiger Shark  (Galeocerdo cuvier)                   82
White Shark (Carcharodon carcharias)               74
Blue Shark (Prionace glauca)                       27
Hammerhead Shark (Sphyrna)                         18
Mako Shark (Isurus oxyrinchus)                     18
Olive Ridley Turtle (Lepidochelys olivacea)        16
Blacktip Shark (Carcharhinus limbatus)              9
Loggerhead Sea Turtle (Caretta caretta)             9
Bull Shark (Carcharhinus leucas)                    4
Guadalupe Fur Seals (Arctocephalus townsendi)       4
Silky Shark (Carcharhinus falciformis)              4
Whale Shark (Rhincodon Typus)                       3
American alligator (Alligator mississippiensis)     2
Green Sea Turtle (Chelonia mydas)                   1
Ship (Motor Vessel)                                 1
 Harbor Seal (Phoca vitulina)                       1
Dolphin (Delphinus capensis)                        1
Pilot Whale (Globicephala)                          1
Name: species, dtype: int64

In [5]:
df = df[df.species.fillna('').str.contains('shark', case=False)]
df.shape

(239, 30)

##### Extract ping data

In [6]:
ping_frames = []
for row in df.itertuples():
    ping_frame = pd.DataFrame(row.pings)
    ping_frame['id'] = row.id
    ping_frames.append(ping_frame)
    
len(ping_frames)

239

##### Merge shark/ping data

In [7]:
pings = pd.concat(ping_frames)
pings.shape

(65938, 6)

In [8]:
joined = pings.merge(df[columns], on='id')
joined.shape

(65938, 13)

In [9]:
joined.head()

,active,datetime,id,latitude,longitude,tz_datetime,name,gender,species,weight,length,tagDate,dist_total
0,1,6 July 2014 1:57:28 PM,3,-34.60661,21.15244,6 July 2014 1:57:28 PM +0900,Oprah,Female,White Shark (Carcharodon carcharias),686 lb,9 ft 10 in.,7 March 2012,2816.662
1,1,23 June 2014 11:40:09 AM,3,-34.78752,19.42479,23 June 2014 11:40:09 AM +0900,Oprah,Female,White Shark (Carcharodon carcharias),686 lb,9 ft 10 in.,7 March 2012,2816.662
2,1,15 June 2014 10:15:44 PM,3,-34.42487,21.09754,15 June 2014 10:15:44 PM +0900,Oprah,Female,White Shark (Carcharodon carcharias),686 lb,9 ft 10 in.,7 March 2012,2816.662
3,1,3 June 2014 11:23:57 AM,3,-34.70432271674724,20.21013441406251,3 June 2014 11:23:57 AM +0900,Oprah,Female,White Shark (Carcharodon carcharias),686 lb,9 ft 10 in.,7 March 2012,2816.662
4,1,29 May 2014 4:53:57 AM,3,-34.65556,19.37459,29 May 2014 4:53:57 AM +0900,Oprah,Female,White Shark (Carcharodon carcharias),686 lb,9 ft 10 in.,7 March 2012,2816.662


##### Clean data

In [10]:
df = joined # don't need a reference to the original resp.json() df anymore
df.shape

(65938, 13)

In [11]:
def clean_weight(value):
    if not value:
        return value
    # most values are like "123 lb"
    value = str(value)
    for character in 'lbs,+':
        value = value.replace(character, '')
    return float(value)

def clean_length(value):
    if not value:
        return value
    # most length values are like '3 ft 4 in.'
    value = str(value)
    total = 0
    if 'ft' in value:
        ft, inches = value.split('ft')
        total += int(ft.strip()) * 12
    else:
        inches = value
    if inches.strip():
        total += float(inches.strip().split()[0])
    return total

df['weight'] = df.weight.apply(clean_weight)
df['length'] = df.length.apply(clean_length)
df['datetime'] = pd.to_datetime(df.tz_datetime)

numeric_cols = ['latitude', 'longitude', 'dist_total', 'weight', 'length']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, axis=1)
df = df.drop(columns=['tz_datetime'])
df.head()

,active,datetime,id,latitude,longitude,name,gender,species,weight,length,tagDate,dist_total
0,1,2014-07-06 13:57:28+09:00,3,-34.606610,21.152440,Oprah,Female,White Shark (Carcharodon carcharias),686.0,118.0,7 March 2012,2816.662
1,1,2014-06-23 11:40:09+09:00,3,-34.787520,19.424790,Oprah,Female,White Shark (Carcharodon carcharias),686.0,118.0,7 March 2012,2816.662
2,1,2014-06-15 22:15:44+09:00,3,-34.424870,21.097540,Oprah,Female,White Shark (Carcharodon carcharias),686.0,118.0,7 March 2012,2816.662
3,1,2014-06-03 11:23:57+09:00,3,-34.704323,20.210134,Oprah,Female,White Shark (Carcharodon carcharias),686.0,118.0,7 March 2012,2816.662
4,1,2014-05-29 04:53:57+09:00,3,-34.655560,19.374590,Oprah,Female,White Shark (Carcharodon carcharias),686.0,118.0,7 March 2012,2816.662


### Explore data

In [12]:
# are all the names unique?
counts = df.groupby('name').id.nunique()
counts.head()

name
AB          1
ANZAC       1
Adelaide    1
Al          1
Albert      1
Name: id, dtype: int64

In [13]:
counts[counts > 1]

Series([], Name: id, dtype: int64)

### Challenge Questions

#### Easy Questions
 1. How many total pings are in the Ocearch shark data?
 2. How many unique species of sharks are in the data set?
 3. What is the name, weight, and species of the heaviest shark(s)?
 4. When and where was the very first ping?
 5. Excluding results with 0 distance traveled: what's the minimum, average, and maximum travel distances?

##### Total pings
How many total pings are in the Ocearch shark data?

In [14]:
len(df)

65938

##### Unique species
How many unique species of sharks are in the data set?

In [15]:
df.species.nunique()

9

##### Heaviest shark(s)
What is the name, weight, and species of the heaviest shark(s)?

In [16]:
max_weight = df.weight.max()
max_weight

25000.0

In [17]:
heavy_sharks = df[df.weight == df.weight.max()]
heavy_sharks.drop_duplicates('name')

,active,datetime,id,latitude,longitude,name,gender,species,weight,length,tagDate,dist_total
56499,1,2016-10-24 12:12:24+09:00,233,38.53890,-68.82060,Rocky Mazzanti,Female,Whale Shark (Rhincodon Typus),25000.0,300.0,24 August 2016,1753.524
59342,1,2017-11-01 09:11:15+09:00,253,40.97405,-69.55461,Canyon,Male,Whale Shark (Rhincodon Typus),25000.0,360.0,9 August 2017,2093.576


##### First ping
When and where was the very first ping?

In [18]:
first_pings = df.sort_values('datetime').head(5)
first_pings[['name', 'datetime', 'latitude', 'longitude']]

,name,datetime,latitude,longitude
519,Oprah,2012-03-10 00:35:31+09:00,-34.132,22.123
518,Oprah,2012-03-10 06:30:24+09:00,-34.160,22.180
517,Oprah,2012-03-10 06:42:45+09:00,-34.158,22.182
516,Oprah,2012-03-10 10:53:02+09:00,-34.165,22.179
752,Albertina,2012-03-10 17:23:50+09:00,-34.179,22.417


##### Distance travelled
Excluding results with 0 distance traveled: what's the minimum, average, and maximum travel distances?

In [19]:
df.dist_total[df.dist_total > 0].describe()

count    65930.000000
mean     12574.930868
std      12749.650916
min          8.127000
25%       3048.274000
50%       8177.352000
75%      17811.853000
max      46553.182000
Name: dist_total, dtype: float64

#### Intermediate questions

 1. Which shark had the most pings?
 2. Which shark has been pinging the longest, and how long has that been?
 3. Which shark species has the most individual sharks tagged?
 4. What is the average length and weight of each shark species?
 5. Which shark has the biggest geographic box (largest distance from min lat/lon to max lat/lon, not dist_traveled)?

##### Most pings
Which shark had the most pings?

In [20]:
groups = df.groupby('id')
sizes = groups.size()
names = groups.name.first()
species = groups.species.first()
first_ping = groups.datetime.min()
last_ping = groups.datetime.max()
combined = pd.concat([sizes, names, species, first_ping, last_ping], axis=1).reset_index()
combined.columns = ['id', 'ping_count', 'name', 'species', 'first_ping', 'last_ping']
combined.sort_values('ping_count', ascending=False).head()

,id,ping_count,name,species,first_ping,last_ping
35,41,3240,Mary Lee,White Shark (Carcharodon carcharias),2012-09-18 18:34:28+09:00,2017-06-17 19:54:32+09:00
36,56,2946,Lydia,White Shark (Carcharodon carcharias),2013-03-03 17:03:13+09:00,2017-03-15 11:31:34+09:00
154,202,2366,Oscar,Mako Shark (Isurus oxyrinchus),2016-07-09 09:14:38+09:00,2019-01-30 05:32:35+09:00
40,60,2134,April,Mako Shark (Isurus oxyrinchus),2013-07-29 02:00:04+09:00,2014-06-17 20:17:03+09:00
26,32,1851,Lisha,White Shark (Carcharodon carcharias),2012-05-15 00:43:21+09:00,2014-04-03 21:48:57+09:00


##### Longest duration pinger
Which shark has been pinging the longest, and how long has that been?

In [21]:
combined['duration'] = combined.last_ping - combined.first_ping
combined.sort_values('duration', ascending=False).head()

,id,ping_count,name,species,first_ping,last_ping,duration
45,65,1816,Katharine,White Shark (Carcharodon carcharias),2013-08-21 13:42:26+09:00,2019-01-15 08:49:00+09:00,1972 days 19:06:34
2,5,204,Helen,White Shark (Carcharodon carcharias),2012-03-11 00:15:10+09:00,2017-01-05 14:22:39+09:00,1761 days 14:07:29
35,41,3240,Mary Lee,White Shark (Carcharodon carcharias),2012-09-18 18:34:28+09:00,2017-06-17 19:54:32+09:00,1733 days 01:20:04
36,56,2946,Lydia,White Shark (Carcharodon carcharias),2013-03-03 17:03:13+09:00,2017-03-15 11:31:34+09:00,1472 days 18:28:21
19,25,1578,Cyndi,White Shark (Carcharodon carcharias),2012-04-15 00:50:25+09:00,2015-09-22 00:00:43+09:00,1254 days 23:10:18


##### Individual count by species
Which shark species has the most individual sharks tagged?

In [22]:
df.groupby('species').id.nunique().sort_values(ascending=False).head()

species
Tiger Shark  (Galeocerdo cuvier)        82
White Shark (Carcharodon carcharias)    74
Blue Shark (Prionace glauca)            27
Mako Shark (Isurus oxyrinchus)          18
Hammerhead Shark (Sphyrna)              18
Name: id, dtype: int64

##### Average length/weight by species
What is the average length and weight of each shark species?

In [23]:
groups = df.groupby('species').agg({'weight' : 'mean', 
                                    'length' : 'mean', 
                                    'id' : 'nunique'})
groups.rename(columns={'id' : 'shark_count'}).sort_values('shark_count', ascending=False)

,weight,length,shark_count
species,,,
Tiger Shark (Galeocerdo cuvier),467.614831,119.147305,82
White Shark (Carcharodon carcharias),1554.920832,147.124877,74
Blue Shark (Prionace glauca),243.634091,106.028852,27
Hammerhead Shark (Sphyrna),126.592041,93.813838,18
Mako Shark (Isurus oxyrinchus),241.199527,82.496546,18
Blacktip Shark (Carcharhinus limbatus),138.378910,80.316209,9
Bull Shark (Carcharhinus leucas),290.400000,89.781022,4
Silky Shark (Carcharhinus falciformis),132.881671,76.965197,4
Whale Shark (Rhincodon Typus),25000.000000,327.906977,3


##### Biggest geographic box
Which shark has the biggest geographic box (largest area calculated from min lat/lon to max lat/lon, not dist_traveled)?

In [24]:
groups = df.groupby('id')
combined = pd.concat([groups.latitude.min(), 
                      groups.longitude.min(), 
                      groups.latitude.max(), 
                      groups.longitude.max(), 
                      groups.name.first(), 
                      groups.species.first()], axis=1).reset_index()
combined.columns = ['id', 'min_lat', 'min_lon', 'max_lat', 'max_lon', 'name', 'species']
combined.head()

,id,min_lat,min_lon,max_lat,max_lon,name,species
0,3,-34.88268,19.374590,-34.05394,22.64236,Oprah,White Shark (Carcharodon carcharias)
1,4,-36.70300,20.535038,-34.06300,22.74626,Albertina,White Shark (Carcharodon carcharias)
2,5,-37.23623,18.536350,-19.50057,37.84922,Helen,White Shark (Carcharodon carcharias)
3,6,-34.98600,19.061580,-24.77363,34.84301,Brenda,White Shark (Carcharodon carcharias)
4,7,-35.46100,17.916810,-32.74300,27.97646,Madiba,White Shark (Carcharodon carcharias)


In [25]:
combined['lat_diff'] = combined.max_lat - combined.min_lat
combined['lon_diff'] = combined.max_lon - combined.min_lon
combined['area'] = combined['lat_diff'] * combined['lon_diff']
combined.sort_values('area', ascending=False).head()

,id,min_lat,min_lon,max_lat,max_lon,name,species,lat_diff,lon_diff,area
29,35,-41.37174,18.51500,-6.15888,71.09830,Kathryn,White Shark (Carcharodon carcharias),35.21286,52.58330,1851.608381
36,56,23.53902,-81.38180,53.65843,-27.48272,Lydia,White Shark (Carcharodon carcharias),30.11941,53.89908,1623.408489
24,30,-43.21756,8.06196,-19.11709,66.72966,Vindication,White Shark (Carcharodon carcharias),24.10047,58.66770,1413.919144
19,25,-45.61157,18.23305,-14.95129,61.87323,Cyndi,White Shark (Carcharodon carcharias),30.66028,43.64018,1338.020138
30,36,-38.82461,17.47565,-10.52038,62.65514,Success,White Shark (Carcharodon carcharias),28.30423,45.17949,1278.770676


#### Hard questions

 1. Use folium to plot the first ping, last ping, and a line connecting each ping for the Tiger shark Emma.  Make the first ping marker a 'play' icon, and last ping icon a 'stop' icon.
 2. Resample Emma data to have a daily lat/lon average, and interpolate missing results.  Plot a marker for each day, and color them blue for hard data, green for interpolated lat/lons
 3. Resample all shark data for daily lat/lon averages, and interpolate missing results
 4. Calculate distance between Emma and other sharks on a daily basis
 5. Identify the shark that has the shortest average distance to Emma per day (minimum 50 days of pings with Emma)
 6. Plot Emma and her closest buddy: interpolated results for each in green, Emma as circle icons and her buddy as square icons

##### Plot Emma locations
Plot the ping locations for the shark named Emma as a `PolyLine` in folium.  Include the first and last ping location as markers.

In [26]:
emma = df.query("name == 'Emma'").sort_values('datetime')
emma.head()

,active,datetime,id,latitude,longitude,name,gender,species,weight,length,tagDate,dist_total
34080,1,2014-02-01 07:10:18+09:00,102,-0.466747,-90.300050,Emma,Female,Tiger Shark (Galeocerdo cuvier),NaN,99.0,20 January 2014,4368.906
34079,1,2014-02-01 07:51:31+09:00,102,-0.411010,-90.327830,Emma,Female,Tiger Shark (Galeocerdo cuvier),NaN,99.0,20 January 2014,4368.906
34078,1,2014-02-01 08:49:34+09:00,102,-0.478080,-90.368890,Emma,Female,Tiger Shark (Galeocerdo cuvier),NaN,99.0,20 January 2014,4368.906
34077,1,2014-02-01 09:25:07+09:00,102,-0.240960,-89.920682,Emma,Female,Tiger Shark (Galeocerdo cuvier),NaN,99.0,20 January 2014,4368.906
34076,1,2014-02-01 17:31:34+09:00,102,-0.429350,-89.649420,Emma,Female,Tiger Shark (Galeocerdo cuvier),NaN,99.0,20 January 2014,4368.906


In [27]:
latlons = list(zip(emma.latitude.values, emma.longitude.values))
latlons[:5]

[(-0.4667466227579547, -90.30004977783199),
 (-0.41101000000000004, -90.32783),
 (-0.47808, -90.36889000000001),
 (-0.24095957143994035, -89.92068179687499),
 (-0.42935, -89.64941999999999)]

In [28]:
import folium

avglat = emma.latitude.mean()
avglon = emma.longitude.mean()

m = folium.Map(location=(avglat, avglon), zoom_start=5)

folium.Marker(latlons[0], popup="First ping", icon=folium.Icon(icon="play")).add_to(m)
folium.Marker(latlons[-1], popup="Last ping", icon=folium.Icon(icon="stop")).add_to(m)

folium.PolyLine(latlons).add_to(m)
    
m

##### Plot interpolated locs
Resample the Emma locations on a per-day basis and interpolate missing locations.  Then, plot the daily markes in folium along with a `PolyLine`.

In [29]:
resampler = emma.set_index('datetime').resample('1D')
resampler

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

In [30]:
averages = resampler[['latitude', 'longitude']].mean()
averages.tail()

,latitude,longitude
datetime,,
2014-08-25 00:00:00+09:00,0.18386,-80.49963
2014-08-26 00:00:00+09:00,0.27332,-80.90630
2014-08-27 00:00:00+09:00,1.35886,-81.55316
2014-08-28 00:00:00+09:00,NaN,NaN
2014-08-29 00:00:00+09:00,2.10503,-80.81086


In [31]:
averages['interpolated'] = averages.latitude.isnull()
averages.tail()

,latitude,longitude,interpolated
datetime,,,
2014-08-25 00:00:00+09:00,0.18386,-80.49963,False
2014-08-26 00:00:00+09:00,0.27332,-80.90630,False
2014-08-27 00:00:00+09:00,1.35886,-81.55316,False
2014-08-28 00:00:00+09:00,NaN,NaN,True
2014-08-29 00:00:00+09:00,2.10503,-80.81086,False


In [32]:
averages.fillna(averages.interpolate(), inplace=True)
averages.tail()

,latitude,longitude,interpolated
datetime,,,
2014-08-25 00:00:00+09:00,0.183860,-80.49963,False
2014-08-26 00:00:00+09:00,0.273320,-80.90630,False
2014-08-27 00:00:00+09:00,1.358860,-81.55316,False
2014-08-28 00:00:00+09:00,1.731945,-81.18201,True
2014-08-29 00:00:00+09:00,2.105030,-80.81086,False


In [33]:
avglat = averages.latitude.mean()
avglon = averages.longitude.mean()

m = folium.Map(location=(avglat, avglon), zoom_start=5)

for row in averages.itertuples():
    if row.interpolated:
        color = 'green'
    else:
        color = 'blue'
    folium.Marker([row.latitude, row.longitude], icon=folium.Icon(color=color)).add_to(m)

m

##### Resample all shark data
Resample all shark data for daily lat/lon averages, and interpolate missing results

In [34]:
data = []

groups = df.groupby('name')
for label, frame in groups:
    resampler = frame.set_index('datetime').resample('1D')
    averages = resampler[['latitude', 'longitude']].mean()
    averages['interpolated'] = averages.latitude.isnull()
    averages.fillna(averages.interpolate(), inplace=True)
    averages['name'] = label
    data.append(averages)
    
resampled = pd.concat(data)
resampled.shape

(63734, 4)

In [35]:
resampled = resampled.set_index('name', append=True).sort_values('datetime')
resampled.head(20)

latitude  longitude  interpolated
datetime                  name                                          
2012-03-10 00:00:00+09:00 Albertina  -34.230000  22.432667         False
                          Brenda     -34.430000  21.594000         False
                          Oprah      -34.147000  22.155333         False
2012-03-11 00:00:00+09:00 Helen      -34.414667  22.416333         False
                          Oprah      -34.079000  22.269000         False
                          Madiba     -34.166500  23.120000         False
                          Brenda     -34.407167  21.043500         False
                          Albertina  -34.540400  22.398000         False
2012-03-12 00:00:00+09:00 Oprah      -34.134000  22.191000         False
                          Madiba     -34.269500  23.319500         False
                          Brenda     -34.391000  20.997000         False
                          Helen      -34.396125  22.323125         False
                          Albertina  -35.685556  22.196778         False
2012-03-13 00:00:00+09:00 Helen      -34.137667  22.167333         False
                          Oprah      -34.374667  21.807000         False
                          Madiba     -34.133667  23.827000         False
                          Albertina  -36.349400  21.521200         False
                          Brenda     -34.392000  20.990500          True
2012-03-14 00:00:00+09:00 Oprah      -34.443000  21.303333         False
                          Albertina  -36.558750  20.814750         False

##### Distance to Emma
Identify the shark that has the shortest average distance to Emma per day

In [36]:
emma = resampled.query('name == "Emma"')
emma.head()

C:\Users\kafon\Anaconda2\envs\py36\lib\site-packages\pandas\core\generic.py:407: FutureWarning: The default of the 'keep_tz' keyword will change to True in a future release. You can set 'keep_tz=True' to obtain the future behaviour and silence this warning.
  s = level_values.to_series()


,,latitude,longitude,interpolated
datetime,name,,,
2014-02-01 00:00:00+09:00,Emma,-0.422161,-90.121436,False
2014-02-02 00:00:00+09:00,Emma,-0.487691,-90.294378,False
2014-02-03 00:00:00+09:00,Emma,-0.454803,-90.324300,False
2014-02-04 00:00:00+09:00,Emma,-0.439116,-90.286372,False
2014-02-05 00:00:00+09:00,Emma,-0.461627,-90.336575,False


In [37]:
emma_locs = emma.reset_index(level='name')
emma_locs.head()

,name,latitude,longitude,interpolated
datetime,,,,
2014-02-01 00:00:00+09:00,Emma,-0.422161,-90.121436,False
2014-02-02 00:00:00+09:00,Emma,-0.487691,-90.294378,False
2014-02-03 00:00:00+09:00,Emma,-0.454803,-90.324300,False
2014-02-04 00:00:00+09:00,Emma,-0.439116,-90.286372,False
2014-02-05 00:00:00+09:00,Emma,-0.461627,-90.336575,False


In [38]:
emma_locs = emma_locs.drop(columns=['name', 'interpolated'])\
                     .rename(columns={'latitude' : 'emma_lat', 'longitude' : 'emma_lon'})
emma_locs.head()

,emma_lat,emma_lon
datetime,,
2014-02-01 00:00:00+09:00,-0.422161,-90.121436
2014-02-02 00:00:00+09:00,-0.487691,-90.294378
2014-02-03 00:00:00+09:00,-0.454803,-90.324300
2014-02-04 00:00:00+09:00,-0.439116,-90.286372
2014-02-05 00:00:00+09:00,-0.461627,-90.336575


In [39]:
joined = resampled.reset_index(level='name').join(emma_locs)
joined.head()

,name,latitude,longitude,interpolated,emma_lat,emma_lon
datetime,,,,,,
2012-03-10 00:00:00+09:00,Albertina,-34.230000,22.432667,False,NaN,NaN
2012-03-10 00:00:00+09:00,Brenda,-34.430000,21.594000,False,NaN,NaN
2012-03-10 00:00:00+09:00,Oprah,-34.147000,22.155333,False,NaN,NaN
2012-03-11 00:00:00+09:00,Helen,-34.414667,22.416333,False,NaN,NaN
2012-03-11 00:00:00+09:00,Oprah,-34.079000,22.269000,False,NaN,NaN


In [40]:
joined[joined.emma_lat.notnull()].head()

,name,latitude,longitude,interpolated,emma_lat,emma_lon
datetime,,,,,,
2014-02-01 00:00:00+09:00,Beamer,11.340995,-76.061746,False,-0.422161,-90.121436
2014-02-01 00:00:00+09:00,Mary Lee,31.432270,-73.988629,True,-0.422161,-90.121436
2014-02-01 00:00:00+09:00,Cyndi,-35.102921,21.927046,True,-0.422161,-90.121436
2014-02-01 00:00:00+09:00,Rizzilient,34.368468,-42.617839,True,-0.422161,-90.121436
2014-02-01 00:00:00+09:00,Guayasamin,-0.476127,-90.243839,False,-0.422161,-90.121436


In [41]:
# calculate distance like a triangle hypotenuse, a**2 + b**2 = c**2
joined['diff_lat'] = joined.latitude - joined.emma_lat
joined['diff_lon'] = joined.longitude - joined.emma_lon
joined['distance'] = (joined.diff_lat**2 + joined.diff_lon**2) ** .5
joined.head()

,name,latitude,longitude,interpolated,emma_lat,emma_lon,diff_lat,diff_lon,distance
datetime,,,,,,,,,
2012-03-10 00:00:00+09:00,Albertina,-34.230000,22.432667,False,NaN,NaN,NaN,NaN,NaN
2012-03-10 00:00:00+09:00,Brenda,-34.430000,21.594000,False,NaN,NaN,NaN,NaN,NaN
2012-03-10 00:00:00+09:00,Oprah,-34.147000,22.155333,False,NaN,NaN,NaN,NaN,NaN
2012-03-11 00:00:00+09:00,Helen,-34.414667,22.416333,False,NaN,NaN,NaN,NaN,NaN
2012-03-11 00:00:00+09:00,Oprah,-34.079000,22.269000,False,NaN,NaN,NaN,NaN,NaN


In [42]:
joined[joined.distance.notnull()].head()

,name,latitude,longitude,interpolated,emma_lat,emma_lon,diff_lat,diff_lon,distance
datetime,,,,,,,,,
2014-02-01 00:00:00+09:00,Beamer,11.340995,-76.061746,False,-0.422161,-90.121436,11.763156,14.059689,18.331577
2014-02-01 00:00:00+09:00,Mary Lee,31.432270,-73.988629,True,-0.422161,-90.121436,31.854430,16.132807,35.706753
2014-02-01 00:00:00+09:00,Cyndi,-35.102921,21.927046,True,-0.422161,-90.121436,-34.680760,112.048481,117.292870
2014-02-01 00:00:00+09:00,Rizzilient,34.368468,-42.617839,True,-0.422161,-90.121436,34.790629,47.503597,58.881063
2014-02-01 00:00:00+09:00,Guayasamin,-0.476127,-90.243839,False,-0.422161,-90.121436,-0.053967,-0.122403,0.133772


##### Emma's buddy
Identify the shark that has the shortest average distance to Emma per day (minimum 50 days of pings with Emma)

In [43]:
groups = joined[joined.distance.notnull()].groupby('name').agg({'distance' : ['size', 'mean']})
groups.sort_values(('distance', 'size'), ascending=False).head()

distance            
             size        mean
name                         
Yolanda       210    8.510538
Cyndi         210  110.587290
Philip        210  111.980580
Andre         210  111.926180
Mary Lee      210   30.570177

In [44]:
buddies = groups[groups[('distance', 'size')] >= 50].sort_values(('distance', 'mean')).head()
buddies

distance          
              size      mean
name                        
Emma           210  0.000000
Gisela          59  0.125881
Berlanga        84  0.709297
Judy           100  2.018307
Encantada       78  2.749212

In [45]:
buddies.index

Index(['Emma', 'Gisela', 'Berlanga ', 'Judy', 'Encantada'], dtype='object', name='name')

##### Plot Emma and Buddy
Plot Emma and her closest buddy on folium.  Emma should be blue/green (known/interpolated) and her buddy should be red/black (known/interpolated).

In [46]:
buddy = buddies.index[1]
buddy

'Gisela'

In [47]:
emma = resampled.query('name == "Emma"')
emma.head()

,,latitude,longitude,interpolated
datetime,name,,,
2014-02-01 00:00:00+09:00,Emma,-0.422161,-90.121436,False
2014-02-02 00:00:00+09:00,Emma,-0.487691,-90.294378,False
2014-02-03 00:00:00+09:00,Emma,-0.454803,-90.324300,False
2014-02-04 00:00:00+09:00,Emma,-0.439116,-90.286372,False
2014-02-05 00:00:00+09:00,Emma,-0.461627,-90.336575,False


In [48]:
buddy = resampled.query('name == "%s"' % buddy)
buddy.head()

,,latitude,longitude,interpolated
datetime,name,,,
2014-01-29 00:00:00+09:00,Gisela,-0.387178,-90.281583,False
2014-01-30 00:00:00+09:00,Gisela,-0.391372,-90.280224,True
2014-01-31 00:00:00+09:00,Gisela,-0.395567,-90.278865,True
2014-02-01 00:00:00+09:00,Gisela,-0.399761,-90.277506,True
2014-02-02 00:00:00+09:00,Gisela,-0.403955,-90.276147,True


In [49]:
avglat = emma.latitude.mean()
avglon = emma.longitude.mean()

m = folium.Map(location=(avglat, avglon), zoom_start=5)

for row in emma.itertuples():
    if row.interpolated:
        color = 'green'
    else:
        color = 'blue'
    folium.Marker([row.latitude, row.longitude], icon=folium.Icon(color=color)).add_to(m)

for row in buddy.itertuples():
    if row.interpolated:
        color = 'black'
    else:
        color = 'red'
    folium.Marker([row.latitude, row.longitude], icon=folium.Icon(color=color)).add_to(m)
    
m